# Project 1, Part 1, Sales Related Queries

University of California, Berkeley

Master of Information and Data Science (MIDS) program

w205 - Fundamentals of Data Engineering

Student:

Year:

Semester:

Section:


# Included Modules and Packages

Code cell containing your includes for modules and packages

In [1]:
import math
import numpy as np
import pandas as pd

import psycopg2

# Supporting code

Code cells containing any supporting code, such as connecting to the database, any functions, etc.  Remember you can use any code from the labs.

In [2]:
#
# function to run a select query and return rows in a pandas dataframe
# pandas puts all numeric values from postgres to float
# if it will fit in an integer, change it to integer
#

def my_select_query_pandas(query, rollback_before_flag, rollback_after_flag):
    "function to run a select query and return rows in a pandas dataframe"
    
    if rollback_before_flag:
        connection.rollback()
    
    df = pd.read_sql_query(query, connection)
    
    if rollback_after_flag:
        connection.rollback()
    
    # fix the float columns that really should be integers
    
    for column in df:
    
        if df[column].dtype == "float64":

            fraction_flag = False

            for value in df[column].values:
                
                if not np.isnan(value):
                    if value - math.floor(value) != 0:
                        fraction_flag = True

            if not fraction_flag:
                df[column] = df[column].astype('Int64')
    
    return(df)
    

In [3]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

# 1.1.1 Total sales as a dollar amount, total number of sales, average dollar amount per sale


Each record in the sales table is an individual sale, and the total_amount column is the total amount for that individual sale.

Write 1 and only 1 query.  Note that the query may have as many subqueries, including "with" clauses, as you wish.  

Name column headers exactly as shown in the example below. 

Format data exactly as shown in the example below.

Ensure that when you check this Juptyer Notebook into GitHub that the query results in the Pandas dataframe are clearly visible in GitHub.

The query should return only 1 row into a Pandas dataframe and should look similar to this: 

||total_sales_dollars|total_sales_million_dollars|total_number_of_sales|average_dollar_amount_per_sale|
|---|---|---|---|---|
|0|98739408|98.7|1537617|64.22|

In [10]:
rollback_before_flag = True
rollback_after_flag = True

query = """
select sum(total_amount) as total_sales_dollars,
       sum(total_amount)/1000000 as total_sales_million_dollars,
       count(sale_id) as total_number_of_sales,
       avg(total_amount) as average_dollar_amount_per_sale
from sales
"""

df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
df

,total_sales_dollars,total_sales_million_dollars,total_number_of_sales,average_dollar_amount_per_sale
0,98739408,98.739408,1537617,64.215866


In [11]:
q = """
select *
from sales
"""
df = my_select_query_pandas(q, rollback_before_flag, rollback_after_flag)
df

,store_id,sale_id,customer_id,sale_date,total_amount
0,1,1,710,2020-01-01,48
1,1,2,1477,2020-01-01,132
2,1,3,1820,2020-01-01,48
3,1,4,1272,2020-01-01,96
4,1,5,1986,2020-01-01,36
...,...,...,...,...,...
1537612,5,227717,29429,2020-12-31,60
1537613,5,227718,31048,2020-12-31,96
1537614,5,227719,30206,2020-12-31,60
1537615,5,227720,29097,2020-12-31,12


# 1.1.2 Total sales as a dollar amount, total number of sales, average dollar amount per sale by store


Each record in the sales table is an individual sale, and the total_amount column is the total amount for that individual sale.

For store_name use the store's city.

Sort by store_name in alphabetical order.

Write 1 and only 1 query.  Note that the query may have as many subqueries, including "with" clauses, as you wish.  

Name column headers exactly as shown in the example below. 

Format data exactly as shown in the example below.

Ensure that when you check this Juptyer Notebook into GitHub that the query results in the Pandas dataframe are clearly visible in GitHub.

The query should return 5 rows into a Pandas dataframe. The first and last rows should look similar to this: 

||store_name|total_sales_dollars|total_sales_million_dollars|total_number_of_sales|average_dollar_amount_per_sale|
|---|---|---|---|---|---|
|0|Berkeley|25041060|25.0|390375|64.15|
|...|...|...|...|...|...|
|4|Seattle|22024512|22.0|342327|64.34|

In [17]:
rollback_before_flag = True
rollback_after_flag = True

query = """
select s.city as store_name,
       sum(sa.total_amount) as total_sales_dollars,
       sum(sa.total_amount)/1000000 as total_sales_million_dollars,
       count(sa.sale_id) as total_number_of_sales,
       avg(sa.total_amount) as average_dollar_amount_per_sale
from stores s
     join sales as sa
     on s.store_id = sa.store_id
group by store_name
order by store_name
"""

df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
df

,store_name,total_sales_dollars,total_sales_million_dollars,total_number_of_sales,average_dollar_amount_per_sale
0,Berkeley,25041060,25.041060,390375,64.146167
1,Dallas,19408260,19.408260,302120,64.240236
2,Miami,17692404,17.692404,275074,64.318707
3,Nashville,14573172,14.573172,227721,63.995732
4,Seattle,22024512,22.024512,342327,64.337642


# 1.1.3 Total sales as a dollar amount, total number of sales, average dollar amount per sale by month

Each record in the sales table is an individual sale, and the total_amount column is the total amount for that individual sale.

Derive the month_number (1 = January) and the month from the sale_date.

Sort by month_number.

Write 1 and only 1 query.  Note that the query may have as many subqueries, including "with" clauses, as you wish.  

Name column headers exactly as shown in the example below. 

Format data exactly as shown in the example below.

Ensure that when you check this Juptyer Notebook into GitHub that the query results in the Pandas dataframe are clearly visible in GitHub.

The query should return 12 rows into a Pandas dataframe. The first and last rows should look similar to this: 

||month_number|month|total_sales_dollars|total_sales_million_dollars|total_number_of_sales|average_dollar_amount_per_sale|
|---|---|---|---|---|---|---|
|0|1|January  |7803828|7.8|121955|63.99|
|...|...|...|...|...|...|...|
|11|12|December |8340420|8.3|130209|64.05|

In [ ]:
rollback_before_flag = True
rollback_after_flag = True

query = """

Replace with your SQL query

"""

df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
df

# 1.1.4 Total sales as a dollar amount, total number of sales, average dollar amount per sale by store and month

Each record in the sales table is an individual sale, and the total_amount column is the total amount for that individual sale.

For store_name use the store's city.

Derive the month_number (1 = January) and the month from the sale_date.

Sort by store_name in alphabetical order then by month_number.

Write 1 and only 1 query.  Note that the query may have as many subqueries, including "with" clauses, as you wish.  

Name column headers exactly as shown in the example below. 

Format data exactly as shown in the example below.

Ensure that when you check this Juptyer Notebook into GitHub that the query results in the Pandas dataframe are clearly visible in GitHub.

The query should return 60 rows into a Pandas dataframe. The first and last rows should look similar to this: 

||store_name|month_number|month|total_sales_dollars|total_sales_million_dollars|total_number_of_sales|average_dollar_amount_per_sale|
|---|---|---|---|---|---|---|---|
|0|Berkeley|1|January  |1988904|2.0|31045|64.07|
|...|...|...|...|...|...|...|...|
|59|Seattle|12|December |1876056|1.9|29136|64.39|

In [ ]:
rollback_before_flag = True
rollback_after_flag = True

query = """

Replace with your SQL query

"""

df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
df

# 1.1.5 Total sales as a dollar amount, total number of sales, average dollar amount per sale by day of week

Each record in the sales table is an individual sale, and the total_amount column is the total amount for that individual sale.

Derive the dow (0 = Sunday) and the day_of_week from the sale_date.

Sort by dow.

Write 1 and only 1 query.  Note that the query may have as many subqueries, including "with" clauses, as you wish.  

Name column headers exactly as shown in the example below. 

Format data exactly as shown in the example below.

Ensure that when you check this Juptyer Notebook into GitHub that the query results in the Pandas dataframe are clearly visible in GitHub.

**Note: the reference output is in Markdown which drops trailing zeros.  Pandas does not drop trailing zeros.  This is ok.**

The query should return 7 rows into a Pandas dataframe. The first and last rows should look similar to this: 

||dow|day_of_week|total_sales_dollars|total_sales_million_dollars|total_number_of_sales|average_dollar_amount_per_sale|
|---|---|---|---|---|---|---|
|0|0|Sunday   |18589068|18.6|289869|64.13|
|...|...|...|...|...|...|...|
|6|6|Saturday |19421460|19.4|302055|64.3|

In [ ]:
rollback_before_flag = True
rollback_after_flag = True

query = """

Replace with your SQL query

"""

df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
df

# 1.1.6 Total sales as a dollar amount, total number of sales, average dollar amount per sale by store and day of week


Each record in the sales table is an individual sale, and the total_amount column is the total amount for that individual sale.

For store_name use the store's city.

Derive the dow (0 = Sunday) and the day_of_week from the sale_date.

Sort by store_name, then by dow.

Write 1 and only 1 query.  Note that the query may have as many subqueries, including with clauses, as you wish.  

Name column headers exactly as shown in the example below. 

Format data exactly as shown in the example below.

Sort data exactly as shown in the example below.

Ensure that when you check this Juptyer Notebook into GitHub that the query results in the Pandas dataframe are clearly visible in GitHub.


The query should return 35 rows into a Pandas dataframe. The first and last rows should look similar to this: 

||store_name|dow|day_of_week|total_sales_dollars|total_sales_million_dollars|total_number_of_sales|average_dollar_amount_per_sale|
|---|---|---|---|---|---|---|---|
|0|Berkeley|0|Sunday   |4694640|4.7|73481|63.89|
|...|...|...|...|...|...|...|...|
|34|Seattle|6|Saturday |4336704|4.3|67220|64.52|

In [ ]:
rollback_before_flag = True
rollback_after_flag = True

query = """

Replace with your SQL query

"""

df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
df